In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from airflow_dag import process_data_with_spark_store_in_mongo

/opt/anaconda3/envs/DistDataSys/lib/python3.10/site-packages/airflow/models/base.py:49 MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)

In [3]:
process_data_with_spark_store_in_mongo()

23/02/27 15:06:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/27 15:06:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
[2023-02-27 15:06:37,294] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_34.182_-118.247_2022_02.csv'
23/02/27 15:06:44 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[2023-02-27 15:06:48,077] {mongo_db.py:44} WARNING - INSERT KEY ERROR: 34.182_-118.247_2022-02-01_00:00:00
[2023-02-27 15:06:48,444] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_34.182_-118.247_2022_03.csv'
[2023-02-27 15:06:51,358] {mongo_db.py:44} WARNING - INSERT KEY ERROR: 34.182_-118.247_2022-03-01_00:00:00
[2023-02-27 15:06:51,665] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_34.182_-118.247_2022_04.csv'
[2023-02-27 15:06:53,975] {mongo_db.py:44} WARNING - INSERT KEY ERROR: 34.182_-118.247_2022-04-01_00:00:00
[2023-02-27 15:06:54,305] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_34.182_-118.247_2022_05.csv'


[2023-02-27 15:06:58,002] {mongo_db.py:44} WARNING - INSERT KEY ERROR: 34.182_-118.247_2022-05-01_00:00:00
[2023-02-27 15:06:58,322] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_34.182_-118.247_2022_06.csv'
[2023-02-27 15:07:00,530] {mongo_db.py:44} WARNING - INSERT KEY ERROR: 34.182_-118.247_2022-06-01_00:00:00
[2023-02-27 15:07:00,855] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_34.182_-118.247_2022_07.csv'


[2023-02-27 15:07:03,296] {mongo_db.py:44} WARNING - INSERT KEY ERROR: 34.182_-118.247_2022-07-01_00:00:00
[2023-02-27 15:07:03,602] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_34.182_-118.247_2022_08.csv'
[2023-02-27 15:07:05,787] {mongo_db.py:44} WARNING - INSERT KEY ERROR: 34.182_-118.247_2022-08-01_00:00:00
[2023-02-27 15:07:06,102] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_34.182_-118.247_2022_09.csv'
[2023-02-27 15:07:08,242] {mongo_db.py:44} WARNING - INSERT KEY ERROR: 34.182_-118.247_2022-09-01_00:00:00
[2023-02-27 15:07:08,571] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_34.182_-118.247_2022_10.csv'
[2023-02-27 15:07:10,342] {mongo_db.py:44} WARNING - INSERT KEY ERROR: 34.182_-118.247_2022-10-01_00:00:00
[2023-02-27 15:07:10,654] {airflow_dag.py:93} INFO - Processing: gcp_filename = 'data/vc_34.182_-118.247_2022_11.csv'
[2023-02-27 15:07:12,761] {mongo_db.py:44} WARNING - INSERT KEY ERROR: 34.182_-118.247_2022-11-01_00